In [1]:
import os
print(os.getcwd())
os.chdir('/home/duynguyenngoc/Desktop/project/analytic/dwh_analytic/')

/home/duynguyenngoc/Desktop/project/analytic/dwh_analytic/dags/data_warehouse_prod


In [2]:
from pprint import pprint
import os
from pymongo import MongoClient
import time
from datetime import date, timedelta
import datetime
from dateutil import tz
import pendulum
from dags.data_warehouse_prod.functions import function as func
from dags.data_warehouse_prod.functions.report import initial_report
from dags.data_warehouse_prod.functions.db_connect import EngineConnect as DatabaseConnect
from dags.data_warehouse_prod.settings import config
from dags.data_warehouse_prod.schema.loading_data import LoadingData
from dags.data_warehouse_prod.schema.backup_data_with_report import BackupData
from dags.data_warehouse_prod.schema.fact_document import FactDocumentModel
from dags.data_warehouse_prod.schema.fact_performance import FactPerformanceModel
from dags.data_warehouse_prod.schema.fact_data_extraction import FactDataExtractionModel
from dags.data_warehouse_prod.schema.dim_project_variable import DimProjectVariableModel

In [3]:
db_connect = DatabaseConnect(uri = config.DWH_SQLALCHEMY_URI)
project_variable = DimProjectVariableModel(project_id = '5e9e7ec598d753001b7efe6b')
project_variable.init_schema(db_connect)
project_variable.init_data(db_connect)

In [ ]:
class LoadingDataModel:
    def __init__(
        self,
        *kwagrs,
        
    )
class ProjectExecutor(LoadingDataModel, BackupDataModel, CleanDataModel, ReportDataModel ,DimProjectVariableModel):
    def __init__(
        self,
        *kwargs,
        environment: str,
        project_id: str,
        environment: str,
        uri: str,
        database_name: str = 'elrond',
        db: DatabaseConnect
    ):
        self.db = db
        self.environment = environment
        self.project_id = project_id
        self.database_name  = database_name
        

    
    def run_backup_docs(self):
        pass
    
    def run_backup_trans(self):
        pass
    
    def run_backup_performance(self):
        pass
    
    def run_backup_field(self):
        pass
    
    def run_report(self):
        pass
    
    def run_dim_field(self):
        pass
    
    def run_fact_document(self):
        pass
    
    def run_fact_perfromance(self):
        pass
    
    def run_fact_data_extraction(self):    
        pass

In [3]:
import pandas as pd

In [14]:
class GdaExecutor(LoadingData, BackupData):
    def __init__(
        self,
        *kwargs,
        environment: str,
        uri: str,
        database_name: str,
        docs_collection_name: str, 
        trans_collection_name: str,
        performance_collection_name: str,
        db: DatabaseConnect
    ):
        self.environment = environment
        self.uri = uri
        self.database_name = database_name
        self.docs_collection_name = docs_collection_name
        self.trans_collection_name = trans_collection_name
        self.performance_collection_name = performance_collection_name
        self.db = db
        self.maxSevSelDelay = 20000
        self.start = config.start
        self.query = config.GDA_QUERY
        self.performance_query = config.GDA_PERFORMANCE_QUERY
        self.project_id = config.GDA_PROJECT_ID
        self.project_name = config.GDA_PROJECT_NAME
        self.backup_dir = config.BACKUP_DIR
        self.project_backup_dir = config.GDA_BACKUP_DIR
        self.project_docs_dir = config.GDA_DOCS_DIR
        self.project_trans_dir = config.GDA_TRANS_DIR
        self.project_field_dir = config.GDA_FIELD_DIR
        self.project_performance_dir = config.GDA_PERFORMANCE_DIR
        self.backup_file_type = config.BACKUP_FILE_TYPE
        self.schema = config.DWH_ANALYTIC_SCHEMA
        self.fact_document_table = config.DWH_FACT_DOCUMENT_TABLE
        self.fact_performance_table = config.DWH_FACT_PERFORMANCE_TABLE
        self.fact_data_extraction_table = config.DWH_FACT_DATA_EXTRACTION_TABLE
        self.fact_report_table = config.DWH_FACT_REPORT_ETL_TABLE
        self.schedule_type = 'daily'
        self.schedule_date_key = 20200113
        self.schedule_time_key = 50000
        self.reports = []
        self.document_key_checks = []
        self.performance_key_checks = []
    
    def run_clean(self):
        report = self.clean()
        self.reports.append(report)
        return report
    
    def run_backup_trans(self):
        report = self.backup_trans()
        self.reports.append(report)
        return report

    def run_backup_docs(self):
        report = self.backup_docs()
        self.reports.append(report)
        return report
        
            
    def run_backup_performance(self):
        report = self.backup_performance()
        self.reports.append(report)
        return report
    
    
    def fact_document(self):
        report = initial_report('fact_document', self.project_id, self.schedule_type, self.schedule_date_key, self.schedule_time_key)
        start_run = time.time()
        try:
            datas = []
            data_docs, data_trans = self.get_docs_and_trans()
            list_created = [{'doc_id': func.bson_object_to_string(data['_id']), 'created_date': data['created_date']} for data in data_docs]
            _id = self.db.get_max_id_table(schema = self.schema, table = self.fact_document_table, col = 'document_key')
            if _id == None:
                _id = 1
            else:
                _id+=1
            _id =1
            for data in data_trans:
                if len(data['records']) == 0:
                    continue
                created_date_utc_7  = func.created_date_of_docs_by_id(func.bson_object_to_string(data['doc_id']), list_created) + datetime.timedelta(hours = 7)
                last_modified_utc_7 = data['last_modified'] + datetime.timedelta(hours = 7)
                import_date_key_utc_7, import_time_key_utc_7 = func.handle_date_to_date_and_time_id(created_date_utc_7)
                export_date_key_utc_7, export_time_key_utc_7 = func.handle_date_to_date_and_time_id(last_modified_utc_7)
                document_id = func.bson_object_to_string(data['doc_id'])
                doc_set_id = func.bson_object_to_string(data['doc_set_id']),
                _obj = FactDocumentModel(
                    document_key = _id,
                    ori_document_id = func.bson_object_to_string(data['_id']),
                    project_id = self.project_id,
                    document_id = document_id,
                    doc_set_id =  doc_set_id,
                    remark_code = None,
                    remark_description = None,
                    import_date_key = import_date_key_utc_7,
                    import_time_key = import_time_key_utc_7,
                    export_date_key = export_date_key_utc_7,
                    export_time_key = export_time_key_utc_7,
                    import_timestamp = created_date_utc_7,
                    export_timestamp = last_modified_utc_7,
                )
                self.document_key_checks.append({'document_key': _id, 'document_id': document_id, 'doc_set_id': doc_set_id})
                datas.append(_obj)
                _id+=1
            self.db.create([item.__dict__ for item in datas], self.schema, self.fact_document_table)
            df = pd.json_normalize([item.__dict__ for item in datas])
            report.status_code = 'PASSED'
        except Exception as e:
            report.status_code = 'FAILED'
            report.description = str(e)
        finally:
            report.total_time_run_second = time.time()-start_run
            self.reports.append(report)
            return report

db_connect = DatabaseConnect(uri = config.DWH_SQLALCHEMY_URI)
executor = GdaExecutor(
    environment=config.ENVIRONMENT,
    uri=config.ELROND_URI,
    database_name=config.ELROND_DATABASE,
    docs_collection_name= config.GDA_DOCS_COLLECTION, 
    trans_collection_name= config.GDA_TRANS_COLLECTION,
    performance_collection_name = config.GDA_PERFORMANCE_COLLECTION,
    db = db_connect
)

# executor.run_clean()
# executor.run_backup_docs()
# executor.run_backup_performance()
# executor.run_backup_trans()
re = executor.fact_document()
pprint(re.__dict__)
# executor.fact_performance()
# executor.fact_data_extract() 
# executor.report_upload()
# executor.show_report()


{'description': '(sqlite3.InterfaceError) Error binding parameter 4 - probably '
                'unsupported type.\n'
                '[SQL: INSERT INTO fact_document (document_key, '
                'ori_document_id, project_id, document_id, doc_set_id, '
                'import_time_key, import_date_key, export_time_key, '
                'export_date_key, import_timestamp, export_timestamp, '
                'remark_code, remark_description) VALUES (?, ?, ?, ?, ?, ?, ?, '
                '?, ?, ?, ?, ?, ?)]\n'
                "[parameters: ((1, '5ffbaffc489b01001ed5a29f', "
                "'5e9e7ec598d753001b7efe6b', '5ff2df89474eb70010c1a5aa', "
                "('5ff2df89474eb70010c1a4de',), 162737, 20210104, 85508, "
                "20210111, '2021-01-04 16:27:37.617000', '2021-01-11 "
                "08:55:08.149000', None, None), (2, "
                "'5ffbaffc489b01001ed5a2a0', '5e9e7ec598d753001b7efe6b', "
                "'5ff2df89474eb70010c1a5ab', ('5ff2df89474eb70010

In [5]:
db_connect = DatabaseConnect(uri = config.DWH_SQLALCHEMY_URI)
executor = GdaExecutor(
    environment=config.ENVIRONMENT,
    uri=config.ELROND_URI,
    database_name=config.ELROND_DATABASE,
    docs_collection_name= config.GDA_DOCS_COLLECTION, 
    trans_collection_name= config.GDA_TRANS_COLLECTION,
    performance_collection_name = config.GDA_PERFORMANCE_COLLECTION,
    db = db_connect
)

# executor.run_clean()
# executor.run_backup_docs()
# executor.run_backup_performance()
# executor.run_backup_trans()
re = executor.fact_document()
pprint(re.__dict__)
# executor.fact_performance()
# executor.fact_data_extract() 
# executor.report_upload()
# executor.show_report()

{'description': '(sqlite3.OperationalError) no such table: '
                'dwh_development_analytic.fact_document\n'
                '[SQL: select max(document_key) from '
                'dwh_development_analytic."fact_document";]\n'
                '(Background on this error at: http://sqlalche.me/e/13/e3q8)',
 'executor_date_key': 2021117,
 'executor_date_timestamp': datetime.datetime(2021, 1, 17, 1, 5, 45, 790863, tzinfo=tzfile('/usr/share/zoneinfo/Asia/Ho_Chi_Minh')),
 'executor_time_key': 1545,
 'job_name': 'fact_document',
 'project_id': '5e9e7ec598d753001b7efe6b',
 'schedule_date_key': 20200113,
 'schedule_time_key': 50000,
 'status_code': 'FAILED',
 'total_time_run_second': 2.618730068206787}


In [7]:
   def get_document_key_by_document_id(self, document_id: str):
        document_key = None
        for item in self.document_key_checks:
            if item['document_id'] == document_id:
                document_key = item['document_key']
                break
        return document_key
    
    def fact_performance(self):
        report = initial_report('fact_performance', self.project_id, self.schedule_type, self.schedule_date_key, self.schedule_time_key)
        start_run = time.time()
        try:
            datas = []
            data_performance = self.get_performance()
            _id = self.db.get_max_id_table(schema = self.schema, table = self.fact_performance_table, col ='performance_key')
            if _id == None:
                _id = 1
            else:
                _id+=1
            for performance in data_performance:
                captured_date_timestamp_utc_7 = performance['time'] + datetime.timedelta(hours = 7)
                document_key = self.get_document_key_by_document_id(performance['doc_id'])
                obj_ = FactPerformanceModel(
                        performance_key = _id,
                        ori_performance_id = func.bson_object_to_string(performance['_id']),
                        document_key = document_key,
                        project_id = self.project_id,  
                        group_id = performance['group_id'],  
                        document_id = performance['doc_id'],  
                        reworked = func.int_to_bool(performance['rework_count']),  
                        work_type_key = func.get_working_type_id_by_name(performance['work_type']),  
                        process_key = func.get_process_key_performance_gda(performance['type'], performance['task_def_key']),  
                        number_of_record = performance['records'],
                        number_of_item = performance['items'],  
                        number_of_field = performance['fields'],
                        number_of_character = performance['chars'],  
                        user_name = performance['username'], 
                        ip = performance['ip'], 
                        captured_date_timestamp = captured_date_timestamp_utc_7,  
                        captured_date_key = func.time_to_date_key(captured_date_timestamp_utc_7),  
                        captured_time_key = func.time_to_time_key(captured_date_timestamp_utc_7),  
                        total_time_second = performance['total_time']    
                )
                datas.append(obj_)
                self.performance_key_checks.append({'performance_key': _id, 'user_name': performance['username'], 
                                                    'document_key': document_key, 'module_type': performance['type'], 
                                                    'task_def_key': performance['task_def_key']})
                _id+=1
            self.db.create([item.__dict__ for item in datas], self.schema, self.fact_performance_table)
            report.status_code = 'PASSED'
        except Exception as e:
            report.status_code = 'FAILED'
            report.description = str(e)
            pprint(e)
        finally:
            report.total_time_run_second = time.time()-start_run
            self.reports.append(report)
            return report
        
    def get_performance_key(self, document_key: str, user_name: str, module_type: str, task_def_key: str):
        performance_key = None
        for item in self.performance_key_checks:
            if item['user_name'] == user_name and item['module_type'] == module_type and item['document_key'] == document_key and item['task_def_key'] == task_def_key:
                performance_key = item['performance_key']
                break
        return performance_key
                                                   
    def fact_data_extract(self):
        report = initial_report('fact_data_extraction', self.project_id, self.schedule_type, self.schedule_date_key, self.schedule_time_key)
        start_run = time.time()
        try:
            data_docs, data_trans = self.get_docs_and_trans()
            col_ignores = ['ImagePath']
            results = []                
            for data in data_docs:
                records = data['records']
                document_id = func.bson_object_to_string(data['_id'])          
                document_key = self.get_document_key_by_document_id(document_id)
                doc_set_id = func.bson_object_to_string(data['doc_set_id'])
                for record in records:
                    for key, value in record.items():
                        if key == 'keyed_data':
                            for keyed_data in value:
                                source = keyed_data['source']
                                task_def_key = keyed_data['task_def_key']
#                                 task_id = keyed_data['task_id']
#                                 section = keyed_data['section']
#                                 reason = keyed_data['reason']
                                data_needed = keyed_data['data'][0].items()
                                last_modified_utc_7 = keyed_data['createdtime'] + datetime.timedelta(hours = 7)
                                user_name = keyed_data['keyer']
                                performance_key = None
                                if source != 'queue_transform' and task_def_key.startswith('Type'):
                                    process_key = 3 # human input keyed_data kpi
                                    performance_key = self.get_performance_key(document_key, user_name, 'keying', task_def_key) 
                                if source != 'queue_transform' and task_def_key == 'Verify_Hold_Type':
                                    process_key = 12 # human check bad_image keyed_data not kpi                               
                                elif source == 'queue_transform' and task_def_key.startswith('Type'):
                                    process_key = 4 # 'machine save input keyed_data'
                                elif source != 'queue_transform' and task_def_key.startswith('Proof'):
                                    process_key = 5 # human qc input keyed_data' kpi
                                    performance_key = self.get_performance_key(document_key, user_name, 'keying', task_def_key)                                  
                                elif source == 'queue_transform' and task_def_key.startswith('Proof'):
                                    process_key = 6 # 'machine save qc keyed_data'
                                for field_name, field_value_dict in data_needed:
                                    if field_name in col_ignores:
                                        continue
                                    _obj = FactDataExtractionModel(
                                        document_key = document_key,
                                        performance_key = performance_key,
                                        ori_document_id = document_id,
                                        project_id = self.project_id,
                                        document_id = document_id,
                                        doc_set_id =  doc_set_id,
                                        last_modified_date_key = func.time_to_date_key(last_modified_utc_7),
                                        last_modified_time_key = func.time_to_time_key(last_modified_utc_7),
                                        last_modified_timestamp = last_modified_utc_7,
                                        user_name = user_name,
                                        process_key = process_key,
                                        field_name = field_name,
                                        field_value = field_value_dict['text']
                                    )
                                    results.append(_obj)

                        elif key == 'final_data':
                            final_data = value[0]
                            data_needed = final_data['data'][0].items()
                            last_modified_utc_7 = final_data['createdtime'] + datetime.timedelta(hours = 7)
                            user_name = final_data['keyer']
                            process_key = 10
                            for field_name, field_value_dict in data_needed:
                                if field_name in col_ignores:
                                    continue
                                _obj = FactDataExtractionModel(
                                    document_key = document_key,
                                    performance_key = None,
                                    ori_document_id = document_id,
                                    project_id = self.project_id,
                                    document_id = document_id,
                                    doc_set_id =  doc_set_id,
                                    last_modified_date_key = func.time_to_date_key(last_modified_utc_7),
                                    last_modified_time_key = func.time_to_time_key(last_modified_utc_7),
                                    last_modified_timestamp = last_modified_utc_7,
                                    user_name = user_name,
                                    process_key = process_key,
                                    field_name = field_name,
                                    field_value = field_value_dict['text']
                                )
                                results.append(_obj)

                        elif key == 'qc_ed_data':
                            qc_ed_data = value[0][0]
                            if 'qc_fields_err' not in qc_ed_data.keys():
                                continue
                            qc_ed_data_err = qc_ed_data['qc_fields_err']
                            data_needed = qc_ed_data_err[0].items()
                            last_modified_utc_7 = qc_ed_data['createdtime'] + datetime.timedelta(hours = 7)
                            user_name = qc_ed_data['keyer']
                            process_key = 8
                            performance_key = None
                            performance_key = self.get_performance_key(document_key, user_name, 'qc', task_def_key) 
                            for field_name, field_value_dict in data_needed:
                                if field_name in col_ignores:
                                    continue
                                _obj = FactDataExtractionModel(
                                    document_key = document_key,
                                    performance_key = performance_key,
                                    ori_document_id = document_id,
                                    project_id = self.project_id,
                                    document_id = document_id,
                                    doc_set_id =  doc_set_id,
                                    last_modified_date_key = func.time_to_date_key(last_modified_utc_7),
                                    last_modified_time_key = func.time_to_time_key(last_modified_utc_7),
                                    last_modified_timestamp = last_modified_utc_7,
                                    user_name = user_name,
                                    process_key = process_key,
                                    field_name = field_name,
                                    field_value = field_value_dict['text']
                                )
                                results.append(_obj)

                        elif key == 'apr_ed_data':
                            report.description = 'Not handle aprove qc data because not have sample data'
#             for data in data_trans:
                

            self.db.create([item.__dict__ for item in results], self.schema, self.fact_data_extraction_table)
            report.status_code = 'PASSED'
        except Exception as e:
            report.status_code = 'FAILED'
            report.description = str(e)
        finally:
            report.total_time_run_second = time.time()-start_run
            self.reports.append(report)
            return report

    def report_upload(self):
        self.db.create([item.__dict__ for item in self.reports], self.schema, self.fact_report_table)
    
    def show_report(self):
        for report in self.reports:
            pprint(report.__dict__)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 9)